# Emoties herkennen

In deze activiteit zal je een AI systeem ontwikkelen dat in staat is om emoties te herkennen. Zo leer je, stap voor stap, verschillende principes van AI en machinaal leren. 

## Voorkennis

Om met deze notebook aan de slag te gaan, heb je een basiskennis nodig van het Python programmeren. We gebruiken in deze notebook datatypes, operatoren, structuren en functies. Ben je niet zeker dat je voldoende Python kennis hebt voor deze notebook, dan kun je terecht op [dwengo.org/python_programming](https://dwengo.org/python_programming/). Daar worden alle basisprincipes stap voor stap uitgelegd.

## Installeren en importeren van de nodige bibliotheken

Voor we starten met ons systeem te bouwen, laden we eerst een aantal bibliotheken in. Deze bevatten voorgeprogrammeerde functies die we in de analyse nodig zullen hebben.

In [ ]:
# Bibliotheken installeren
!pip install opencv-contrib-python

In [ ]:
# Bibliotheken inladen
import matplotlib.pyplot as plt
from PIL import Image
from scripts import helpers
import numpy as np
from sklearn.model_selection import train_test_split

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer

## Data verzamelen

AI-systemen leren regels op basis van data. De kwaliteit van een AI-systeem staat of valt dus ook met de kwaliteit van de dataset. Er zijn een aantal voorwaarden waaraan je dataset moet voldoen voor we die kwalitatief kunnen noemen.

* **Correcte labels**: De informatie in de dataset moet correct zijn. Afbeeldingen van katten moeten het label "kat" krijgen, afbeeldingen van honden, het label "hond". Afbeeldingen die een fout label hebben, zullen het AI-systeem in de war sturen.
* **Volledige informatie**: De datataset moet alle informatie bevatten om het probleem op te lossen. Als je katten en honden wil detecteren, moet je bijvoorbeeld foto's hebben van alle katten- en hondenrassen.
* **Unieke elementen**: De elementen in de dataset moeten uniek zijn. Elke foto van een kat of een hond komt dus maar één keer voor in de dataset. Foto's die meerdere keren voorkomen, helpen niet om het systeem te verbeteren.
* **In evenwicht**: Er zijn evenveel voorbeelden voor elk soort element. Bijvoorbeeld evenveel voorbeelden van katten als van honden.
* **Ethisch**: Ben je op een ethische manier aan de dataset gekomen? Zijn er auteursrechten op de data? Bevat de data persoonlijke gegevens?

In deze activiteit stellen we zelf onze dataset samen. Zo kunnen we waken over de kwaliteit. Je zal merken dat het heel wat werk is om een dataset op te stellen. Het samenstellen van een kwalitatieve dataset is vaak een van de belangrijkste uitdagingen bij het ontwikkelen van een AI-systeem.

### Emoties tekenen

We willen een systeem bouwen dat emoties kan detecteren. Dit doen we niet onmiddelijk op foto's van mensen, daar zouden we een te complex systeem voor nodig hebben. We starten met het detecteren van de emoties van smileys. Specifiek proberen we eerst **lachende en verbaasde smileys** van elkaar te onderscheiden. Hieronder zie je een voorbeeld van een lachende en een verbaasde smiley.

![](images/voorbeeld_blij.png)


![](images/voorbeeld_verbaasd.png)

Om ons AI-systeem te trainen, hebben we een honderdtal blije en een honderdtal verbaasde smileys nodig. Ja, je zal die zelf moeten tekenen en dat is heel wat werk. Geen nood, we hebben een methode voorzien waarop je dat op een gemakkelijke manier kan doen. We gebruiken een sjabloon. Het sjabloon bevat een raster. In elke hokje van het raster teken je een smiley. **Per blad teken je slecht één emotie**, dus ofwel allemaal blije smileys ofwel allemaal verbaasde. Hieronder zie je een voorbeeld van zo'n ingevuld sjabloon.

![](images/voorbeeld_raster_blij.jpg)

**Opdracht**: Drukt het document *raster.pdf* 10 keer af op A3 formaat. Reserveer 5 bladen voor blije smileys en 5 bladen voor verbaasde. Vul het raster in door smileys te tekenen. Tip: je kan de bladen verdelen over verschillende personen, zo verdeel je het tekenwerk.

**Opdracht**: Neem foto's van de ingevulde sjablonen. Zorg ervoor dat de vier markers aan de hoeken van het raster zichtbaar zijn op de foto. Zorg er ook voor dat de foto de smileys en de markers voldoende scherp in beeld brengt.

## De data inladen

Nu je een dataset hebt verzameld, moeten we die inladen in Python. Voeg je afbeeldingen daarvoor links in de bestandsverkenner toe aan het correcte mapje. Je ziet een mapje met de naam *dataset*. Daaring zitten twee submappen met de namen *blij* en *verbaasd*. Voeg de foto's toe aan de correcte map. 

Om een bestand naar die map te uploaden, klik je bovenaan in de bestandsverkenner op het upload icoontje. Op onderstaande afbeelding is dat icoontje aangegeven met een groene pijl.

![](images/hoe_uploaden.png)

We hebben al een aantal functies voorzien die het makkelijker maken om de gegevens te verwerken. Hieronder roepen we een functie op die twee parameters krijgt. De eerste parameter is het mapje met afbeeldingen van blije smileys, de tweede parameter is het label voor de items in die map.

In [ ]:
# We laden alle afbeeldingen in de map 'dataset/blij' in en geven ze de label 'blij'
# De rasters op de afbeeldingen worden automatisch in stukjes geknipt.
afbeeldingen_blij, labels_blij = helpers.laadt_bestanden_in_map_met_label("dataset/blij", label="blij")

Nu we de blije afbeeldingen hebben ingeladen, bekijken we hoe deze eruitzien. In de cel hieronder zie je de code om verschillende eigenschappen van onze dataset weer te geven.

In [ ]:
print(f"De dataset bevat {len(afbeeldingen_blij)} afbeeldingen met label 'blij'")
print(f"De labels zijn: {labels_blij}")
print(f"De eerste afbeelding heeft een grootte van {afbeeldingen_blij[0].shape}")
print("De eerste zes afbeeldingen zien er als volgt uit:")
helpers.toon_afbeeldingen(afbeeldingen_blij, labels_blij, max_afbeeldingen=6)

**Opdracht**: Vul onderstaande codecellen aan zodat je de afbeeldingen van verbaasde smileys opslaat in een variabele. 

In [ ]:
# Vul deze code aan op de plaatsen waar ___ staat.
afbeeldingen_verbaasd, labels_verbaasd = helpers.laadt_bestanden_in_map_met_label("___", label="___")

**Opdracht**: Vul ook onderstaande codecel aan om de informatie over de verbaasde smileys af te drukken.

In [ ]:
# Vul deze code aan op de plaatsen waar ___ staat.
print(f"De dataset bevat {___} afbeeldingen met label '___'")
print(f"De labels zijn: {___}")
print(f"De eerste afbeelding heeft een grootte van {___}")
print("De eerste zes afbeeldingen zien er als volgt uit:")
helpers.toon_afbeeldingen(___, ___, max_afbeeldingen=6)

## De data klaarmaken voor het AI-systeem

Nu we onze afbeeldingen en labels ingeladen hebben in Python, kunnen we deze verwerken tot een formaat dat het AI-systeem nodig heeft. Daarvoor doorlopen we de volgende stappen.
1. We voegen onze blije en verbaasde afbeeldingen samen tot één dataset.
2. De labels omzetten van tekst naar getallen.
3. We splitsen deze dataset op in drie verzamelingen.
    * De trainingsvezameling: deze gebruiken we om ons AI-systeem te trainen.
    * De testverzameling: deze gebruiken we om de prestatie van het AI-systeem tijdens de ontwikkeling te testen. De afbeeldingen in deze verzameling overlappen niet met de trainingsverzameling. Deze verzameling is nodig om te zien of het AI-systeem kan generaliseren en dus niet gewoon de afbeeldingen in de trainingsverzameling vanbuiten geleerd heeft.
    * De validatieverzameling: deze gebruiken we om de prestatie van het AI-systeem na de ontwikkeling te valideren. De afbeeldingen in deze verzameling overlappen niet met die in de train- en testverzamelingen.

Het is waarschijnlijk nog niet helemaal duidelijk waarom we deze verzamelingen nodig hebben. Door deze notebook te doorlopen zou dit duidelijker moeten worden. In de onderstaande cellen gaan we alvast van start met het opstellen van de verzamelingen.


### Stap 1: het samenvoegen van de afbeeldingen en labels

Met de onderstaande code voegen we alle afbeeldignen en labels samen. Het resultaat zijn twee numpy arrays, een met de afbeeldingen en een met de labels.

In [ ]:
afbeeldingen = np.vstack([np.array(afbeeldingen_blij), np.array(afbeeldingen_verbaasd)])
afbeeldingen = np.expand_dims(afbeeldingen, axis=-1)
labels = np.concatenate([np.array(labels_blij), np.array(labels_verbaasd)])

Druk informatie over de arrays af.

In [ ]:
print(f"De dataset bevat {afbeeldingen.shape[0]} afbeeldingen")
print(f"Er zijn {len(labels)} labels")
print(f"De eerste afbeelding heeft een grootte van {afbeeldingen[0].shape}")


**Opdracht**: Controleer het formaat van de dataset. Komt deze overeen met de som van het aantal blije en verbaasde afbeeldingen?

### Stap 2: De labels omzetten van tekst naar getallen.

Omdat computers sneller en efficiënter kunnen rekenen met getallen, zetten we onze labels om van tekst naar getallen. Hier gebruiken we **one-hot** encodering. We zullen elk label voorstellen door twee getallen. Het eerste getal is een 1 wanneer het label *blij* is en een 0 wanneer het label *verbaasd* is. Het tweede getal is een 0 wanneer het label *blij* is en 1 wanneer het label *verbaasd* is. Hieronder zie je een voorbeeld.

![](images/formaat_labels.png)

In [ ]:
# Deze code zal onze labels one-hot encoderen.
labels_one_hot = helpers.one_hot_encode_labels(labels, ["blij", "verbaasd"])

Nu we de nieuwe labels hebben, kunnen we 10 willekeurige afbeeldingen afdrukken met hun nieuwe label.

In [ ]:
# Genereer 10 willekeurige indices.
random_indices = np.random.randint(0, len(labels), 10)
# Toon deze 10 willekeurige afbeeldingen.
helpers.toon_afbeeldingen(afbeeldingen[random_indices], labels_one_hot[random_indices], max_afbeeldingen=10)

### Stap 3: de dataset opsplitsen in train-, test- en validatieverzameling.

Om onze dataset op te splitsen in deze drie verzamelingen gebruiken we de functie *train_test_split* uit de *sklearn* bibliotheek. We gebruiken deze twee keer, eerst om een validatieverzameling op te stellen, daarna om een test- en trainverzameling op te stellen.

#### De validatieverzameling opstellen

Onderstaande codecel zal willekeurig 10% van de dataset selecteren als validatieset. 

In [ ]:
overige_afbeeldingen, validatie_afbeeldingen, overige_labels, validatie_labels = train_test_split(afbeeldingen, labels_one_hot, test_size=0.1)

Bekijk het formaat van de validatieverzameling en de verzameling met overige afbeeldingen.

In [ ]:
print(f"De overige dataset bevat {overige_afbeeldingen.shape[0]} afbeeldingen")
print(f"De validatieverzameling bevat {validatie_afbeeldingen.shape[0]} afbeeldingen")

**Opdracht**: Vul onderstaande code aan zodat de *overige_afbeeldingen* en *overige_labels* worden opgesplitst in trainingsverzameling en testverzameling. 20% van de overige afbeeldingen moet gebruikt worden als testverzameling.

In [ ]:
# Vul deze code aan op de plaatsen waar ___ staat.
train_afbeeldingen, test_afbeeldingen, train_labels, test_labels = train_test_split(___, ___, test_size=___)

In [ ]:
print(f"De trainingsverzameling bevat {train_afbeeldingen.shape[0]} afbeeldingen")
print(f"De testverzameling bevat {test_afbeeldingen.shape[0]} afbeeldingen")

## Het AI-systeem trainen

Nu we onze dataset hebben klaargemaakt, kunnen we ons AI-systeem trainen. Hier zullen we gebruik maken van een neuraal netwerk. Het is niet zo belangrijk dat je nu al weet hoe zo'n systeem werkt. Je kan het neuraal netwerk zien als een doos die de regels leert om de emoties te kunnen herkennen. Deze regels leert het netwerk door naar voorbeelden te kijken.

Hieronder zie je een functie die vastlegt wat de structuur van ons neuraal netwerk is. 

In [ ]:
from keras.layers import BatchNormalization
# Deze functie legt vast wat de structuur is van het AI-model dat we gaan trainen.
def maak_neuraal_netwerk(hoogte_afbeelding, breedte_afbeelding):
    model = Sequential()
    
    model.add(InputLayer(input_shape=(hoogte_afbeelding, breedte_afbeelding, 1)))
    
    model.add(Conv2D(1, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.1))
    
    model.add(Conv2D(2, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.1))
    
    model.add(Conv2D(4, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.1))
    
    model.add(Conv2D(8, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.1))
    
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.1))
    
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))   
    model.add(Dropout(0.1)) 
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

Hieronder roepen we onze functie op om ons model aan te maken.

In [ ]:
model = maak_neuraal_netwerk(afbeeldingen.shape[1], afbeeldingen.shape[2])

Met de *summary* functie kunnen we de details van het model afdrukken. Zo zien we ook hoeveel *parameters* we moeten leren. Hoe meer parameters, hoe complexer ons model dus hoe meer data we nodig hebben om het te trainen.

In [ ]:
model.summary()

Nu kunnen we het model trainen aan de hand van onze dataset. Wanneer je de volgende cel uitvoert, zal je zien dat het netwerk begint te leren op basis van onze trainingsverzameling. Je ziet verschillende informatie. 
* In de hoeveelste *Epoch* we zitten. Dit geeft aan hoeveel keer we de volledige trainigsverzameling als voorbeeld hebben gegeven aan het netwerk.
* De *accuracy* wordt berekend door het aantal correcte voorspellingen te delen door het totaal aantal voorspellingen. Hoe hoger de accuracy, hoe beter de prestatie van het netwerk op de trainingsverzameling.
* De *loss* geeft aan hoeveel de voorspellingen van het netwerk gemiddeld afwijken van de correcte waarde. Hoe hoger de loss hoe slechter de prestatie van het netwerk op de trainingsverzameling. 

In [ ]:
model.fit(train_afbeeldingen, train_labels, epochs=10, batch_size=1)

## Het AI-systeem testen

Nu we ons AI-systeem getraind hebben op onze trainingsverzameling, moeten we het ook testen op de testverzameling. Dit is nodig om te controleren of het model echt geleerd heeft om de eigenschappen van de emoties te herkennen. Of het gewoon de volledige trainingsverzameling uit het hoofd geleerd heeft. Om dit te doen gebruiken we onze testverzameling. Deze bevat afbeeldingen die niet gebruikt zijn om het AI-systeem te trainen. Als het systeem werkt op deze afbeeldingen wil dat zeggen dat het model wel degelijk de kenmerken van de emoties heeft geleerd. We zeggen in dit geval dat het model kan *generaliseren* naar andere voorbeelden.

Om de prestatie van ons AI-systeem te meten zijn er verschillende metingen die we kunnen doen. Een eenvoudige meting is de *accuracy*, deze ken je ook al voor de trainingsverzameling. De accuracy is de verhouding tussen het aantal correct voorspelde afbeeldingen en het totale aantal voorspellingen. In onderstaande cel berekenen we de *accuracy* op de testverzameling.

In [ ]:
# Bereken de accuracy op de testverzameling.
loss, accuracy = model.evaluate(test_afbeeldingen, test_labels)
print(f"Test accuracy: {accuracy}")

De waarde die je hier zal bekomen zal altijd een beetje anders zijn. Deze hangt onder andere af van de kwaliteit van jouw dataset. In onze testen komen we hier vaak aan een accuracy van ongeveer 0.95 ofwel 95%.

De *accuracy* geeft ons een idee van de prestaties van het systeem maar zegt ons niet hoe goed het elk van de groepen kan onderscheiden. Om daar meer zicht op te krijgen, kunnen we de *confusion matrix* bekijken. Deze matrix geeft aan hoeveel afbeeldingen van elke categorie correct voorspeld werden. Hieronder zie je een voorbeeld van een confusion matrix die wij bekomen hebben voor ons model.

![](images/voorbeeld_confusion_matrix.png)

Hierboven zie je dus dat 18 blije afbeeldingen correct werden voorspeld. Twee van de blije afbeeldingen werden fout voorspeld, met het label verbaasd dus. Alle verbaade afbeeldingen werden correct voorspeld. 

Voer onderstaande code uit om een beeld te krijgen van de confusion matrix van het model dat jij trainde.

In [ ]:
# Print de confusion matrix
predictions = model.predict(test_afbeeldingen)
confusion_matrix = helpers.create_confusion_matrix_for_one_hot_encoded_labels(test_labels, predictions, ["blij", "verbaasd"])
helpers.visualize_confussion_matrix_in_heatmap(confusion_matrix, ["blij", "verbaasd"])

We kunnen ook de originele afbeelding samen met de voorspelling weergeven.

In [ ]:
mapped_labels_true = ["blij" if np.argmax(label) == 0 else "verbaasd" for label in test_labels]
mapped_labels_predicted = ["blij" if np.argmax(label) == 0 else "verbaasd" for label in predictions]
mapped_labels_combined = [f"Echt: {mapped_labels_true[i]} \n Voorspeld: {mapped_labels_predicted[i]}" for i in range(len(mapped_labels_true))]
helpers.toon_afbeeldingen(test_afbeeldingen, mapped_labels_combined, max_afbeeldingen=len(test_afbeeldingen))

## Het AI-systeem valideren

Normaalgezien zou je nu al een systeem moeten hebben dat redelijk goed werkt. Het is hier dus niet nodig om aanpassingen te doen aan de parameters van het model. Mocht het model nog niet goed werken, kan je ervoor kiezen om dat model aan te passen zodat het een beter resultaat geeft op de testverzameling. Om zeker te zijn dat je aanpassingen goed werken op andere data, kan je dan de validatieverzameling gebruiken. 

Laten we het resultaat op de validativerzameling toch al eens bekijken.

In [ ]:
# Bereken de accuracy op de testverzameling.
validatie_loss, validatie_accuracy = model.evaluate(validatie_afbeeldingen, validatie_labels)
print(f"Test accuracy: {validatie_accuracy}")

# Print de confusion matrix
predictions = model.predict(validatie_afbeeldingen)
confusion_matrix = helpers.create_confusion_matrix_for_one_hot_encoded_labels(validatie_labels, predictions, ["blij", "verbaasd"])
helpers.visualize_confussion_matrix_in_heatmap(confusion_matrix, ["blij", "verbaasd"])



In [ ]:
# Toon de afbeeldingen met hun voorspelling.
mapped_labels_true = ["blij" if np.argmax(label) == 0 else "verbaasd" for label in validatie_labels]
mapped_labels_predicted = ["blij" if np.argmax(label) == 0 else "verbaasd" for label in predictions]
mapped_labels_combined = [f"Echt: {mapped_labels_true[i]} \n Voorspeld: {mapped_labels_predicted[i]}" for i in range(len(mapped_labels_true))]
helpers.toon_afbeeldingen(validatie_afbeeldingen, mapped_labels_combined, max_afbeeldingen=len(validatie_afbeeldingen))

# Uitdaging

We hebben nu een systeem dat twee emoties kan onderscheiden van elkaar. Je kan dit systeem makkelijk uitbreiden naar drie emoties. Daarvoor moet je eerst extra data verzamelen. Wanneer je deze data hebt, kan je je baseren op bovenstaande code om een nieuw systeem te bouwen dat drie emoties van elkaar kan onderscheiden.

# Uitbreiding: meer dan emoties

Je kan dit systeem in pricipe gebruiken om gelijk welke twee tekeningen van elkaar de onderscheiden. Je zou dus in plaats van afbeeldingen van smileys, tekeningen kunnen maken van katten en honden. Je kan dan hetzelfde systeem gebruiken om deze tekeningen van elkaar te onderscheiden.